In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [1]:
# Import required libraries

import cv2                 # working with, mainly resizing, images
import numpy as np
import os
from random import shuffle # randomizing ordered data
from tqdm import tqdm      # a nice pretty percentage bar for tasks.


In [2]:
# define variables 

TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'
IMG_WIDTH, IMG_HEIGHT = 100, 100 # dimensions of our images.
LR = 1e-3

MODEL_NAME = 'pizzas-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match

In [3]:
# One hot encoding of the target colum
def label_img(img, word_label):
    if word_label == 'Pepperoni': return [1,0]
    elif word_label == 'Sausage': return [0,1]

In [4]:
# Create training data
def create_train_data(directory='', label='', num_of_images=1000):
    training_data = []
    if directory=='':
        directory = TRAIN_DIR
    label_dir = directory+'/'+label
    i=0
    for img in tqdm(os.listdir(label_dir)):
        img_label = label_img(img, label)
        path = os.path.join(label_dir,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        training_data.append([np.array(img),np.array(img_label)])
        i=i+1
        if i>num_of_images:
            break
    return training_data

In [29]:
train_data_Pepperoni = create_train_data('Pepperoni')
train_data_Sausage = create_train_data('Sausage')
train_data = train_data_Pepperoni.append(train_data_Sausage)

100%|██████████| 202/202 [00:00<00:00, 262.46it/s]


In [34]:
train_data = train_data_Pepperoni

In [39]:
shuffle(train_data)

In [42]:
np.save('data/train_data.npy', train_data)

In [ ]:
train_data

In [ ]:
shuffle(training_data)
np.save('data/train_data.npy', training_data)

# Data Augumentation


In [14]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [15]:
# Define the data generation object
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.4,
        height_shift_range=0.4,
        shear_range=0.25,
        zoom_range=0.25,
        rescale=1.01,
        zca_epsilon=1e-6,
        horizontal_flip=True,
        fill_mode='nearest')


In [365]:
os.getcwd()

'/home/datascience/projects/vsoft'

In [366]:
string="data/new_train/Pepperoni/._0_9746.jpeg"
print (string[0:-1])

data/new_train/Pepperoni/._0_9746.jpe


In [16]:
# Function to generate images
def augument_images(source_dir_name, dest_dir_name, num_images):
    for image in tqdm(os.listdir(source_dir_name)):
        img = load_img(source_dir_name+'/'+image)
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
        # the .flow() command below generates batches of randomly transformed images
        # and saves the results to the `preview/` directory
        i = 0
        image_name = image[0:-4]
        for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=dest_dir_name, save_prefix=image_name, save_format='jpeg'):
            i += 1
            if i > num_images:
                break  # otherwise the generator would loop indefinitely


In [378]:
#source_dir_name = '/home/datascience/projects/vsoft/data/test'
pepperoni_source_dir_name = '/home/datascience/projects/vsoft/data/train/Pepperoni'
pepperoni_dest_dir_name = 'data/new_train/Pepperoni/'

In [379]:
augument_images(pepperoni_source_dir_name, pepperoni_dest_dir_name, 4)

100%|██████████| 1667/1667 [21:13<00:00,  1.31it/s]


In [17]:
#source_dir_name = '/home/datascience/projects/vsoft/data/test'
sausage_source_dir_name = '/home/datascience/projects/vsoft/data/train/Sausage'
sausage_dest_dir_name = '/home/datascience/projects/vsoft/data/new_train/Sausage'

In [18]:
augument_images(sausage_source_dir_name,sausage_dest_dir_name,10)


100%|██████████| 202/202 [05:25<00:00,  1.54s/it]


# Data Preperation

In [5]:
source_dir_name = '/home/datascience/projects/vsoft/data/new_train'

In [6]:
pepperoni_data = create_train_data(source_dir_name,'Pepperoni',1000)

 12%|█▏        | 987/8335 [00:02<00:21, 341.57it/s]


In [7]:
sausage_data = create_train_data(source_dir_name,'Sausage',1000)

 44%|████▎     | 969/2221 [00:02<00:03, 352.59it/s]


## Save data for later use

In [8]:
np.save('data/pepperoni_data.npy', pepperoni_data)

In [9]:
np.save('data/sausage_data.npy', sausage_data)

## Segregate the data into train and test

In [6]:
pepperoni_data = np.load('data/sausage_data.npy')
sausage_data = np.load('data/sausage_data.npy')

In [12]:
full_data.shape

(2002, 2)

In [11]:
full_data = np.concatenate((pepperoni_data, sausage_data), axis=0)

In [14]:
shuffle(full_data)

In [15]:
train = full_data[:-1600]
test = full_data[-400:]

In [16]:
np.array(train).shape

(402, 2)

In [17]:
X=[]
Y=[]

In [404]:
for i in train:
    #print(i[0])
    #print(len(i))
    X.append(i[0].reshape(IMG_WIDTH, IMG_HEIGHT,1))
    Y.append(i[1])
    #print(i[0].reshape(IMG_WIDTH, IMG_HEIGHT,1).shape)

In [27]:
np.array(X).shape
np.array(Y).shape

(0,)

In [408]:
X= np.array(X)
Y= np.array(Y)

In [454]:
X_test=[]
Y_test=[]

In [456]:
len(train)

11588

In [410]:
for i in test:
    #print(i[0])
    #print(len(i))
    X_test.append(i[0].reshape(IMG_WIDTH, IMG_HEIGHT,1))
    #print(i[0].reshape(IMG_WIDTH, IMG_HEIGHT,1).shape)

In [413]:
np.array(X_test).shape

(3000, 400, 800, 1)

In [414]:
X_test = np.array(X_test)

In [18]:
X = np.array([i[0] for i in train[0:600]]).reshape(-1,IMG_WIDTH,IMG_HEIGHT,1)
Y = np.array([i[1] for i in train[0:600]])

test_x = np.array([i[0] for i in test[0:90]]).reshape(-1,IMG_WIDTH,IMG_HEIGHT,1)
test_y = [i[1] for i in test[0:90]]

# Convolution Model

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [20]:
K.clear_session()

In [22]:
K.clear_session()
sess = tf.Session()
K.set_session(sess)

In [36]:
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16
num_classes = 2

if K.image_data_format() == 'channels_first':
    input_shape = (1, IMG_WIDTH, IMG_HEIGHT) # For RGB, (3, IMG_WIDTH, IMG_HEIGHT) 
else:
    input_shape = (IMG_WIDTH, IMG_HEIGHT, 1) # For RGB, (IMG_WIDTH, IMG_HEIGHT, 3) 



In [40]:
# GPU DEFAULT

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [33]:
with tf.device('/cpu:0'):
    #x = tf.placeholder(tf.float32, shape=(None, 784))
    # shared model living on CPU:0
    # it won't actually be run during training; it acts as an op template
    # and as a repository for shared variables
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

In [41]:
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [42]:
history2 = model.fit(X, Y, verbose=0, epochs=1)

InternalError: Dst tensor is not initialized.
	 [[Node: training_4/Adam/Const_6 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [30976,100] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'training_4/Adam/Const_6', defined at:
  File "/usr/anaconda2/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/anaconda2/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/anaconda2/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/anaconda2/envs/py36/lib/python3.6/site-packages/zmq/eventloop/ioloop.p